<a href="https://colab.research.google.com/github/hosseinkhoshdel/Project1/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget https://www.kaggle.com/datasets/emmarex/plantdisease/download -O plantdisease.zip

!unzip plantdisease.zip

--2023-10-16 06:15:42--  https://www.kaggle.com/datasets/emmarex/plantdisease/download
Resolving www.kaggle.com (www.kaggle.com)... 35.244.233.98
Connecting to www.kaggle.com (www.kaggle.com)|35.244.233.98|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /account/login?titleType=dataset-downloads&showDatasetDownloadSkip=False&messageId=datasetsWelcome&returnUrl=%2Fdatasets%2Femmarex%2Fplantdisease%3Fresource%3Ddownload [following]
--2023-10-16 06:15:43--  https://www.kaggle.com/account/login?titleType=dataset-downloads&showDatasetDownloadSkip=False&messageId=datasetsWelcome&returnUrl=%2Fdatasets%2Femmarex%2Fplantdisease%3Fresource%3Ddownload
Reusing existing connection to www.kaggle.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘plantdisease.zip’

plantdisease.zip        [ <=>                ]   4.80K  --.-KB/s    in 0s      

2023-10-16 06:15:43 (33.5 MB/s) - ‘plantdisease.zip’ saved [4913]

Archive:  p

In [ ]:

import cv2
import numpy as np
import tensorflow as tf
from glob import glob
from tensorflow import keras
from tensorflow.keras import layers

# پیش‌پردازش تصاویر
images_path = 'plantvillage/**/*.jpg'
preprocessed_path = 'preprocessed/'

for img_path in glob(images_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  img = cv2.equalizeHist(img)
  img = cv2.GaussianBlur(img, (3,3), 0)
  edges = cv2.Canny(img, 100, 200)

  save_path = preprocessed_path + img_path.split('/')[-1]
  cv2.imwrite(save_path, edges)


# بارگذاری تصاویر پیش‌پردازش شده
preprocessed_images_path = 'preprocessed/**/*.jpg'

train_data = []
for img_path in glob(preprocessed_images_path):
  img = cv2.imread(img_path)
  train_data.append(img)

# تقسیم بندی داده‌های آموزش و تست
x_train = train_data[:80]
x_test = train_data[80:]

# تعریف و آموزش مدل
model = keras.Sequential([
   layers.Conv2D(32, 3, activation='relu', input_shape=(256, 256, 1)),
   layers.MaxPool2D(),
   layers.Conv2D(64, 3, activation='relu'),
   layers.MaxPool2D(),
   layers.Conv2D(128, 3, activation='relu'),
   layers.MaxPool2D(),
   layers.Flatten(),
   layers.Dense(512, activation='relu'),
   layers.Dense(3)
])

model.compile(optimizer='adam', loss='mse')
model.fit(x_train, y_train, epochs=10)

# تبدیل تصاویر جدید
imgs_3d = model.predict(x_test)

In [ ]:
استخراج ویژگی

In [ ]:
import numpy as np
from skimage.measure import marching_cubes, mesh_surface_area

# بارگذاری مدل های سه بعدی
models_3d = imgs_3d

features = []

for model in models_3d:

  # محاسبه حجم
  volume = np.sum(model)

  # محاسبه سطح
  verts, faces, norm, val = marching_cubes(model, 0)
  surface_area = mesh_surface_area(verts, faces)

  # استخراج ویژگی بافت
  texture = measure_texture(model)

  # استخراج ویژگی شکل
  shape = measure_shape(model)

  # محاسبه میانگین رنگ
  color = np.mean(model, axis=(0,1))

  # ذخیره ویژگی‌های استخراج شده
  features.append([volume, surface_area, texture, shape, color])

# ذخیره ویژگی‌ها در Numpy Array
features = np.asarray(features)
np.save('features.npy', features)